This notebook implements a camera wrapper-style functionality with the new Source & SourcePhotometricExposure objects 

In [1]:
from syotools.models import Camera, Telescope, Spectrograph, Source, SourcePhotometricExposure
import numpy as np, astropy.units as u 
from syotools.defaults import default_camera

Warning, 5 of 20000 bins contained negative fluxes; they have been set to zero.


/Users/tumlinson/anaconda3/envs/hwotools/lib/python3.12/site-packages/pysynphot/units.py:824: RuntimeWarning: divide by zero encountered in log
  return -1.085736 * N.log(arg) + ABZERO


Warning, 3590 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 293 of 2920 bins contained negative fluxes; they have been set to zero.
Warning, 5 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 3590 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 293 of 2920 bins contained negative fluxes; they have been set to zero.


In [19]:
def camera_snr(telescope, template, magnitude, exptime, silent=False): 
      ''' Run a basic SNR calculation that takes in a telescope, 
      spectral template, normalization magnitude, and exptime   
      to compute SNR. For converting magnitude, template, 
      and SNR to exptime, use camera_exptime.py 
      
        usage: 
           snr, hri = camera_snr(telescope, template, mag, snr_goal) 

          positional arguments:

           1-telescope = 'EAC1', 'EAC2', or 'EAC3'. This argument is a string. 
             EAC1 = 6 m inner diameter, 7.2 outer diameter hex pattern, off-axis 
             EAC2 = 6 m diameter off-axis 
             EAC3 = 8 m diameter on-axis 

           2-template = your choice of spectral template: 
               'Classical T Tauri', 'M1 Dwarf', 'G Dwarf', '10 Myr Starburst', 'QSO', 
               'Seyfert 1', 'Seyfert 2', 'Liner', 'O5V Star', 'G2V Star', 'Orion Nebula', 
               'G191B2B (WD)', 'GD71 (WD)', 'GD153 (WD)', 'Starburst, No Dust', 
               'Starburst, E(B-V) = 0.6', 'B5V Star', 'M2V Sta', 'Elliptical Galaxy', 
               'Sbc Galaxy', 'Starburst Galaxy', 'NGC 1068', 'Galaxy with f_esc, HI=1, HeI=1', 
               'Galaxy with f_esc, HI=0.001, HeI=1', 'Blackbody5000', 'Blackbody100000' 

           3-mag = V magnitude to normalize the template spectrum, a float. 

           4-exptime =  desired exptime in hours 

           outputs are arrays with the SNR in each band for FUV, NUV, U, B, V, R, I, J, H, K 
           and the camera object "hri" 
       '''

    # create a Telescope, Camera, and Exposure 
    tel, hri = Telescope(), Camera()
    tel.set_from_json(telescope)

    source = Source() 
    redshift = 0. # changes to these are not implemented yet 
    extinction = 0. 

    source.set_sed(template, magnitude, redshift, extinction, 'galex,fuv')   

    exp = SourcePhotometricExposure() 
    exp.source = source
    
    exp.exptime = [[exptime, exptime, exptime, exptime, exptime, exptime, exptime, exptime, exptime, exptime], 'hr']
    exp.unknown = 'snr'
    tel.add_camera(hri)
    hri.add_exposure(exp)

    print('------ Computing SNR as the Unknown -------') 
    if not silent: 
        for bb, ss in zip(hri.bandnames, exp.snr): print("{}, SNR = {}".format(bb, ss)) 
            
    return exp.snr, exp 

In [20]:
snr, exp = camera_snr('EAC1', 'Flat (AB)', 30., 10.) 

------ Computing SNR as the Unknown -------
FUV, SNR = 18.132480240201964 electron(1/2)
NUV, SNR = 20.565561321786003 electron(1/2)
U, SNR = 26.615271546048564 electron(1/2)
B, SNR = 28.15460119302201 electron(1/2)
V, SNR = 23.738448242220112 electron(1/2)
R, SNR = 17.54591280954839 electron(1/2)
I, SNR = 12.292267626580712 electron(1/2)
J, SNR = 19.757438864350615 electron(1/2)
H, SNR = 16.056524269168886 electron(1/2)
K, SNR = 0.8451195448212911 electron(1/2)


In [26]:
def camera_exptime(telescope, template, magnitude, snrgoal, silent=False): 
    ''' Run a basic SNR calculation that takes in a telescope, 
      spectral template, normalization magnitude, and SNR goal  
      to compute exptime. For converting magnitude, template, 
      and exptime to SNR, use camera_snr.py 
      
        usage: 
          exptime, hri = camera_exptime(telescope, template, mag, snr_goal) 

          positional arguments:

           1-telescope = 'EAC1', 'EAC2', or 'EAC3'. This argument is a string. 
             EAC1 = 6 m inner diameter, 7.2 outer diameter hex pattern, off-axis 
             EAC2 = 6 m diameter off-axis 
             EAC3 = 8 m diameter on-axis 

           2-template = your choice of spectral template: 
               'Classical T Tauri', 'M1 Dwarf', 'G Dwarf', '10 Myr Starburst', 'QSO', 
               'Seyfert 1', 'Seyfert 2', 'Liner', 'O5V Star', 'G2V Star', 'Orion Nebula', 
               'G191B2B (WD)', 'GD71 (WD)', 'GD153 (WD)', 'Starburst, No Dust', 
               'Starburst, E(B-V) = 0.6', 'B5V Star', 'M2V Sta', 'Elliptical Galaxy', 
               'Sbc Galaxy', 'Starburst Galaxy', 'NGC 1068', 'Galaxy with f_esc, HI=1, HeI=1', 
               'Galaxy with f_esc, HI=0.001, HeI=1', 'Blackbody5000', 'Blackbody100000' 

           3-mag = V magnitude to normalize the template spectrum, a float.

           4-snr_goal = desired SNR, per pixel, for each band 

           outputs are arrays with the exptime in each band for FUV, NUV, U, B, V, R, I, J, H, K 
       '''
    # create a Telescope, Camera, and Exposure 
    tel, hri = Telescope(), Camera()
    tel.set_from_json(telescope)

    source = Source() 
    redshift = 0. # changes to these are not implemented yet 
    extinction = 0. 

    source.set_sed(template, magnitude, redshift, extinction, 'galex,fuv')   
        
    exp = SourcePhotometricExposure() 
    exp.source = source

    exp._snr = [snr_goal] * u.Unit('electron(1/2)')  
    exp.unknown = 'exptime' 
    tel.add_camera(hri)
    hri.add_exposure(exp)

    print('-- Computing Exptime as the Unknown --') 
    if not silent: 
        for bb, ee in zip(hri.bandnames, exp.exptime): print("{}, SNR = {}".format(bb, ee)) 

    return exp.exptime, hri 

In [27]:
snr, exp = camera_exptime('EAC1', 'QSO', 30., 8.) 

-- Computing Exptime as the Unknown --
FUV, SNR = 8868.584623391833 s
NUV, SNR = 5533.9690121392805 s
U, SNR = 3770.853845507179 s
B, SNR = 3454.728937863258 s
V, SNR = 5086.295890000656 s
R, SNR = 5556.9301520684 s
I, SNR = 8640.368796941711 s
J, SNR = 1496.1976133696564 s
H, SNR = 1449.4766706861926 s
K, SNR = 204615.8781487314 s


In [33]:
def camera_magnitude(telescope, template, snr, exptime, silent=False): 
    ''' Run a basic SNR calculation that takes in a telescope, 
      spectral template, normalization magnitude, and SNR goal  
      to compute exptime. For converting magnitude, template, 
      and exptime to SNR, use camera_snr.py 
      
        usage: 
          exptime, hri = camera_magnitude(telescope, template, mag, snr_goal) 

          positional arguments:

           1-telescope = 'EAC1', 'EAC2', or 'EAC3'. This argument is a string. 
             EAC1 = 6 m inner diameter, 7.2 outer diameter hex pattern, off-axis 
             EAC2 = 6 m diameter off-axis 
             EAC3 = 8 m diameter on-axis 

           2-template = your choice of spectral template: 
               'Classical T Tauri', 'M1 Dwarf', 'G Dwarf', '10 Myr Starburst', 'QSO', 
               'Seyfert 1', 'Seyfert 2', 'Liner', 'O5V Star', 'G2V Star', 'Orion Nebula', 
               'G191B2B (WD)', 'GD71 (WD)', 'GD153 (WD)', 'Starburst, No Dust', 
               'Starburst, E(B-V) = 0.6', 'B5V Star', 'M2V Sta', 'Elliptical Galaxy', 
               'Sbc Galaxy', 'Starburst Galaxy', 'NGC 1068', 'Galaxy with f_esc, HI=1, HeI=1', 
               'Galaxy with f_esc, HI=0.001, HeI=1', 'Blackbody5000', 'Blackbody100000' 

           3-snr_goal = desired SNR, per pixel, for each band 

           4-exptime = exposure time per band in hours

           outputs are arrays with the limiting in each band for FUV, NUV, U, B, V, R, I, J, H, K 
       '''
    
    # create a Telescope, Camera, and Exposure 
    tel, hri = Telescope(), Camera()
    tel.set_from_json(telescope)

    source = Source() 
    redshift = 0. # changes to these are not implemented yet 
    extinction = 0. 

    source.set_sed(template, 30., redshift, extinction, 'galex,fuv')   
        
    exp = SourcePhotometricExposure() 
    exp.source = source

    exp.exptime = [[exptime, exptime, exptime, exptime, exptime, exptime, exptime, exptime, exptime, exptime], 'hr']
    exp._snr = [snr] * u.Unit('electron(1/2)')  
    
    exp.unknown = 'magnitude' 
    tel.add_camera(hri)
    hri.add_exposure(exp)

    print('--- Computing Magnitude as the Unknown ---') 
    if not silent: 
        for bb, mm in zip(hri.bandnames, exp.magnitude): print("{}, SNR = {}".format(bb, mm)) 

    return exp.magnitude, hri 

In [34]:
snr, exp = camera_magnitude('EAC1', 'QSO', 5., 20.) 

--- Computing Magnitude as the Unknown ---
FUV, SNR = 31.92760465093968 mag(AB)
NUV, SNR = 32.088187008511454 mag(AB)
U, SNR = 32.36688518681279 mag(AB)
B, SNR = 32.40787628277046 mag(AB)
V, SNR = 32.17539766499057 mag(AB)
R, SNR = 31.802149654283163 mag(AB)
I, SNR = 31.40465166373087 mag(AB)
J, SNR = 31.903590551778652 mag(AB)
H, SNR = 31.666327050325386 mag(AB)
K, SNR = 28.446163302739667 mag(AB)
